In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./input_data/titanic.csv", index_col='PassengerId')

## general info about data

In [3]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## count male and female

In [5]:
gender = df["Sex"].value_counts()

w = open("./task_1/answer1.dat", "w")
w.write(f"{gender.male} {gender.female}")
w.close()

## count survived

In [6]:
answer = np.round(100 * df["Survived"].sum() / df["Survived"].count(), 2) 

w = open("./task_1/answer2.dat", "w")
w.write(f"{answer}")
w.close()

## passenger class count

In [7]:
tmp = 100 * df["Pclass"].value_counts() / df["Pclass"].count()
out = tmp.round(2).loc[1]

w = open("./task_1/answer3.dat", "w")
w.write(f"{out}")
w.close()

## age: mean vs median

In [8]:
w = open("./task_1/answer4.dat", "w")
w.write(f"{np.round(df.Age.mean(), 2)} {np.round(df.Age.median(),2)}")
w.close()

## correlation

In [9]:
out = df.corr()["Parch"]["SibSp"].round(2)

w = open("./task_1/answer5.dat", "w")
w.write(f"{out}")
w.close()

## pick female surnames

In [10]:
df1 = df[df["Sex"]=='female']

In [11]:
df1["Name"].head()

PassengerId
2     Cumings, Mrs. John Bradley (Florence Briggs Th...
3                                Heikkinen, Miss. Laina
4          Futrelle, Mrs. Jacques Heath (Lily May Peel)
9     Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
10                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [12]:
[name for name in df1["Name"].tolist() if "Mrs." not in name if "Miss." not in name]

['Aubart, Mme. Leontine Pauline',
 'Reynaldo, Ms. Encarnacion',
 'Duff Gordon, Lady. (Lucille Christiana Sutherland) ("Mrs Morgan")',
 'Sagesser, Mlle. Emma',
 'Mayne, Mlle. Berthe Antonine ("Mrs de Villiers")',
 'Rothes, the Countess. of (Lucy Noel Martha Dyer-Edwards)',
 'Leader, Dr. Alice (Farnham)']

## the surname is the first word after the title or the first word in the brackets

In [13]:
d_names = df1["Name"].to_dict()

surname = {}

for idx,name in d_names.items():
    if "(" in name:
        tmp = name.split("(")[-1].split()[0]
        surname[idx] = tmp if ")" not in tmp else tmp.rstrip(")")
    if "Mrs" in name:
        if idx not in surname:
            surname[idx] = name.split("Mrs. ")[-1].split()[0]
    if "Miss" in name:
        if idx not in surname:
            surname[idx] = name.split("Miss. ")[-1].split()[0]
    if "Mme." in name:
        if idx not in surname:
            surname[idx] = name.split("Mme. ")[-1].split()[0]
    if "Ms." in name:
        if idx not in surname:
            surname[idx] = name.split("Ms. ")[-1].split()[0]
    if "Mlle." in name:
        surname[idx] = name.split("Mlle. ")[-1].split()[0]
    if "Dr." in name:
        surname[idx] = name.split("Dr. ")[-1].split()[0]

In [14]:
df2 = pd.Series(surname)

## Transfer as a new feature

In [15]:
df1["Surname"] = df2

/home/laptop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
## check if joining is OK

df1["Surname"].isna().sum()

0

In [17]:
df1[["Surname", "Name"]].head()

,Surname,Name
PassengerId,,
2,Florence,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
3,Laina,"Heikkinen, Miss. Laina"
4,Lily,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
9,Elisabeth,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"
10,Adele,"Nasser, Mrs. Nicholas (Adele Achem)"


In [18]:
df1["Surname"].value_counts().head()

Anna         15
Mary         14
Elizabeth    11
Margaret     10
Alice         7
Name: Surname, dtype: int64

In [19]:
df1[df1["Surname"]=="Anna"]["Name"]

PassengerId
23                           McGowan, Miss. Anna "Annie"
107                     Salkjelsvik, Miss. Anna Kristine
129                                    Peter, Miss. Anna
142                             Nysten, Miss. Anna Sofia
168      Skoog, Mrs. William (Anna Bernhardina Karlsson)
248                      Hamalainen, Mrs. William (Anna)
259                                     Ward, Miss. Anna
301             Kelly, Miss. Anna Katherine "Annie Kate"
313                Lahtinen, Mrs. William (Anna Sylfven)
367     Warren, Mrs. Frank Manley (Anna Sophia Atkinson)
424    Danbom, Mrs. Ernst Gilbert (Anna Sigrid Maria ...
678                              Turja, Miss. Anna Sofia
748                                Sinkkonen, Miss. Anna
766                 Hogeboom, Mrs. John C (Anna Andrews)
787                            Sjoblom, Miss. Anna Sofia
Name: Name, dtype: object

## Anna and Annie are different surnames -> Mary is the most frequent

In [20]:
w = open("./task_1/answer6.dat", "w")
w.write("Mary")
w.close()